In [29]:
execfile("setup_fraud.txt")

In [30]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [31]:
df_x=df.drop(["fraud_reported","auto_make","auto_model"], axis=1, inplace=False)
df_x=pd.get_dummies(df_x)
df_y=df["fraud_reported"]
df_train_x, df_test_x, df_train_y, df_test_y = train_test_split(df_x, df_y, test_size=0.3, random_state=1234)

print("train data X size:",df_train_x.shape)
print("train data Y size:",df_train_y.shape)
print("test data X size:",df_test_x.shape)
print("test data Y size:",df_test_y.shape)

('train data X size:', (700, 110))
('train data Y size:', (700L,))
('test data X size:', (300, 110))
('test data Y size:', (300L,))


In [32]:
rf_uncustomized = RandomForestClassifier(random_state=1234)
rf_uncustomized.fit(df_train_x, df_train_y)
print("Accuracy on training set: {:.3f}".format(rf_uncustomized.score(df_train_x,df_train_y)))
print("Accuracy on test set: {:.3f}".format(rf_uncustomized.score(df_test_x,df_test_y)))

Accuracy on training set: 0.987
Accuracy on test set: 0.773
